In [1]:
import pandas as pd
import numpy as np

In [6]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision',150)

In [8]:
df = pd.read_json('../data/whats-cooking/train.json')

In [23]:
def lower_list(arr):
    return [x.lower() for x in arr]

In [ ]:
def contains_ingredient(item_id):
    ings = df[df['id'] == item_id]['ingredients']
    ings_list = list(ings)[0]
    for item in ings_list:
        df.loc[df['id'] == item_id, [item]] = 1
    

In [ ]:
df['id'].map(contains_ingredient)

In [ ]:
df.head()

In [24]:
df['ingredients'] = df['ingredients'].map(lower_list)

In [25]:
df['ingredients'].head(30)

0     [romaine lettuce, black olives, grape tomatoes...
1     [plain flour, ground pepper, salt, tomatoes, g...
2     [eggs, pepper, salt, mayonaise, cooking oil, g...
3                   [water, vegetable oil, wheat, salt]
4     [black pepper, shallots, cornflour, cayenne pe...
5     [plain flour, sugar, butter, eggs, fresh ginge...
6     [olive oil, salt, medium shrimp, pepper, garli...
7     [sugar, pistachio nuts, white almond bark, flo...
8     [olive oil, purple onion, fresh pineapple, por...
9     [chopped tomatoes, fresh basil, garlic, extra-...
10    [pimentos, sweet pepper, dried oregano, olive ...
11    [low sodium soy sauce, fresh ginger, dry musta...
12    [italian parsley leaves, walnuts, hot red pepp...
13    [ground cinnamon, fresh cilantro, chili powder...
14    [fresh parmesan cheese, butter, all-purpose fl...
15    [tumeric, vegetable stock, tomatoes, garam mas...
16    [greek yogurt, lemon curd, confectioners sugar...
17    [italian seasoning, broiler-fryer chicken,

In [17]:
df['ingredient_count'] = df['ingredients'].map(lambda x: len(x))